In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [12]:
origin_df=pd.read_csv('https://production-bobsim.s3.ap-northeast-2.amazonaws.com/public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv', encoding='euc-kr')

In [13]:
origin_df.head()

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),최소 상대습도(pct),평균 상대습도(pct),합계 일조시간(hr),합계 일사량(MJ/m2)
0,90,속초,2014-01-01,6.6,4.1,8.9,NaN,NaN,9.8,5.4,22.0,31.3,7.0,NaN
1,90,속초,2014-01-02,4.0,-0.3,7.0,NaN,NaN,4.2,1.8,18.0,31.9,8.9,NaN
2,90,속초,2014-01-03,6.6,2.4,9.4,NaN,NaN,9.6,3.0,24.0,40.0,4.4,NaN
3,90,속초,2014-01-04,1.2,0.0,3.8,18.75,7.4,6.0,1.8,44.0,87.4,0.0,NaN
4,90,속초,2014-01-05,0.4,-2.0,2.7,0.83,0.0,5.1,1.7,78.0,90.0,3.7,NaN


### distribution

In [15]:
origin_df.describe()

,지점,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),최소 상대습도(pct),평균 상대습도(pct),합계 일조시간(hr),합계 일사량(MJ/m2)
count,214325.000000,214086.000000,214300.000000,214305.000000,22288.000000,77862.000000,214217.000000,214143.000000,214246.000000,213919.000000,213966.000000,92353.000000
mean,197.148312,12.939258,8.211564,18.397884,6.013794,8.989293,4.815078,2.025076,43.862217,68.523449,6.317802,13.500785
std,64.042405,9.583057,10.119724,9.682490,5.655850,19.070405,2.181304,1.382729,19.651868,16.153310,3.934152,7.162765
min,90.000000,-19.400000,-25.200000,-15.200000,0.000000,0.000000,0.000000,0.000000,1.000000,10.400000,0.000000,0.000000
25%,137.000000,4.700000,-0.300000,10.200000,1.420000,0.100000,3.400000,1.100000,28.000000,57.600000,2.800000,8.000000
50%,201.000000,13.700000,8.400000,19.600000,4.170000,1.600000,4.400000,1.700000,43.000000,70.100000,7.200000,12.700000
75%,258.000000,21.200000,17.000000,26.500000,9.080000,9.100000,5.700000,2.500000,57.000000,80.500000,9.400000,18.770000
max,295.000000,34.100000,30.300000,41.000000,24.000000,384.300000,49.000000,21.400000,100.000000,100.000000,50.000000,42.860000


### Null & Type check

In [8]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214325 entries, 0 to 214324
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   지점             214325 non-null  int64  
 1   지점명            214325 non-null  object 
 2   일시             214325 non-null  object 
 3   평균기온(°C)       214086 non-null  float64
 4   최저기온(°C)       214300 non-null  float64
 5   최고기온(°C)       214305 non-null  float64
 6   강수 계속시간(hr)    22288 non-null   float64
 7   일강수량(mm)       77862 non-null   float64
 8   최대 풍속(m/s)     214217 non-null  float64
 9   평균 풍속(m/s)     214143 non-null  float64
 10  최소 상대습도(pct)   214246 non-null  float64
 11  평균 상대습도(pct)   213919 non-null  float64
 12  합계 일조시간(hr)    213966 non-null  float64
 13  합계 일사량(MJ/m2)  92353 non-null   float64
dtypes: float64(11), int64(1), object(2)
memory usage: 22.9+ MB


In [9]:
origin_df[['지점', '합계 일사량(MJ/m2)']].groupby("지점")['합계 일사량(MJ/m2)'].isnull().count().value_counts().sum()

AttributeError: 'SeriesGroupBy' object has no attribute 'isnull'

In [39]:
origin_df[['지점', '합계 일사량(MJ/m2)']].groupby("지점")['합계 일사량(MJ/m2)'].count().value_counts()

0       38
1        8
2272     8
2270     7
2271     5
2268     5
2        4
2266     3
2269     2
691      1
3        1
526      1
402      1
2199     1
2213     1
1270     1
1600     1
2249     1
1613     1
1268     1
2258     1
2267     1
2159     1
1265     1
2257     1
Name: 합계 일사량(MJ/m2), dtype: int64

### Null value %

In [40]:
origin_df.isnull().sum()/214325* 100 

지점                0.000000
지점명               0.000000
일시                0.000000
평균기온(°C)          0.111513
최저기온(°C)          0.011665
최고기온(°C)          0.009332
강수 계속시간(hr)      89.600840
일강수량(mm)         63.671060
최대 풍속(m/s)        0.050391
평균 풍속(m/s)        0.084918
최소 상대습도(pct)      0.036860
평균 상대습도(pct)      0.189432
합계 일조시간(hr)       0.167503
합계 일사량(MJ/m2)    56.909833
dtype: float64

## null handling
#### 강수 계속시간 , 일강수량, 합계일사량 -> 0
#### 나머지 -> mean()

In [4]:
origin_df[['일강수량(mm)','강수 계속시간(hr)']] = origin_df[['일강수량(mm)','강수 계속시간(hr)']].fillna(value=0)

origin_def.assign()

In [5]:
origin_df['합계 일사량(MJ/m2)'] = origin_df['합계 일사량(MJ/m2)'].fillna(value=0)

In [ ]:
origin_df[['평균기온(°C)','최저기온(°C)','최고기온(°C)', '강수 계속시간(hr)', '일강수량(mm)', '최대 풍속(m/s)',        
          '평균 풍속(m/s)', '최소 상대습도(pct)', '평균 상대습도(pct)', '합계 일조시간(hr)']]=origin_df[['평균기온(°C)','최저기온(°C)','최고기온(°C)', '강수 계속시간(hr)', '일강수량(mm)', '최대 풍속(m/s)',        
          '평균 풍속(m/s)', '최소 상대습도(pct)', '평균 상대습도(pct)', '합계 일조시간(hr)']].fillna(origin_df.mean())

In [6]:
origin_df.isnull().sum()

지점                 0
지점명                0
일시                 0
평균기온(°C)         239
최저기온(°C)          25
최고기온(°C)          20
강수 계속시간(hr)        0
일강수량(mm)           0
최대 풍속(m/s)       108
평균 풍속(m/s)       182
최소 상대습도(pct)      79
평균 상대습도(pct)     406
합계 일조시간(hr)      359
합계 일사량(MJ/m2)      0
dtype: int64

### 합계 일사량은 조사자체를 안하는 지역이 많아서 null value가 많다 ( 삭제 )

In [7]:
origin_df.drop('합계 일사량(MJ/m2)', axis = 1)

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),최소 상대습도(pct),평균 상대습도(pct),합계 일조시간(hr)
0,90,속초,2014-01-01,6.6,4.1,8.9,0.00,0.0,9.8,5.4,22.0,31.3,7.0
1,90,속초,2014-01-02,4.0,-0.3,7.0,0.00,0.0,4.2,1.8,18.0,31.9,8.9
2,90,속초,2014-01-03,6.6,2.4,9.4,0.00,0.0,9.6,3.0,24.0,40.0,4.4
3,90,속초,2014-01-04,1.2,0.0,3.8,18.75,7.4,6.0,1.8,44.0,87.4,0.0
4,90,속초,2014-01-05,0.4,-2.0,2.7,0.83,0.0,5.1,1.7,78.0,90.0,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214320,295,남해,2020-03-17,11.2,3.0,18.1,0.00,0.0,4.1,2.0,29.0,44.4,10.3
214321,295,남해,2020-03-18,11.6,4.4,18.8,0.00,0.0,4.5,1.5,24.0,60.1,10.3
214322,295,남해,2020-03-19,12.5,7.9,18.3,0.00,0.0,6.7,3.0,18.0,48.4,9.6
214323,295,남해,2020-03-20,11.1,6.4,16.6,0.00,0.0,3.9,1.9,24.0,36.0,10.0


** 숫자 피처들에 대한 데이터 분포 왜곡도 확인 후 높은 왜곡도를 가지는 피처 추출 **

In [9]:
from scipy.stats import skew

features_index = origin_df.dtypes[origin_df.dtypes != 'object'].index

skew_features = origin_df[features_index].apply(lambda x : skew(x))

skew_features_top = skew_features[skew_features > 1]
print(skew_features_top.sort_values(ascending=False))
#print(skew_features.sort_values(ascending=False))

일강수량(mm)         8.058130
강수 계속시간(hr)      5.292071
합계 일사량(MJ/m2)    1.171629
dtype: float64


In [10]:
origin_df.describe()

,지점,평균기온(°C),최저기온(°C),최고기온(°C),강수 계속시간(hr),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),최소 상대습도(pct),평균 상대습도(pct),합계 일조시간(hr),합계 일사량(MJ/m2)
count,214325.000000,214086.000000,214300.000000,214305.000000,214325.000000,214325.000000,214217.000000,214143.000000,214246.000000,213919.000000,213966.000000,214325.000000
mean,197.148312,12.939258,8.211564,18.397884,0.625384,3.265715,4.815078,2.025076,43.862217,68.523449,6.317802,5.817511
std,64.042405,9.583057,10.119724,9.682490,2.587713,12.280545,2.181304,1.382729,19.651868,16.153310,3.934152,8.173439
min,90.000000,-19.400000,-25.200000,-15.200000,0.000000,0.000000,0.000000,0.000000,1.000000,10.400000,0.000000,0.000000
25%,137.000000,4.700000,-0.300000,10.200000,0.000000,0.000000,3.400000,1.100000,28.000000,57.600000,2.800000,0.000000
50%,201.000000,13.700000,8.400000,19.600000,0.000000,0.000000,4.400000,1.700000,43.000000,70.100000,7.200000,0.000000
75%,258.000000,21.200000,17.000000,26.500000,0.000000,0.400000,5.700000,2.500000,57.000000,80.500000,9.400000,11.210000
max,295.000000,34.100000,30.300000,41.000000,24.000000,384.300000,49.000000,21.400000,100.000000,100.000000,50.000000,42.860000


** 왜곡도가 1인 피처들은 로그 변환 적용 **

In [ ]:
import numpy as np
origin_df[skew_features_top.index] = np.log1p(origin_df[skew_features_top.index])

In [30]:
skew_features = origin_df[features_index].apply(lambda x : skew(x))
print(skew_features.sort_values(ascending=False))

강수 계속시간(hr)      3.536727
일강수량(mm)         2.196918
평균 풍속(m/s)       0.764090
합계 일사량(MJ/m2)    0.507175
최대 풍속(m/s)       0.332204
지점              -0.440484
최소 상대습도(pct)    -0.613731
합계 일조시간(hr)     -1.063731
평균 상대습도(pct)    -1.088218
평균기온(°C)              NaN
최저기온(°C)              NaN
최고기온(°C)              NaN
dtype: float64
